In [16]:
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation, Trainer, TrainingArguments,  Mask2FormerImageProcessor
from PIL import Image
import requests
import torch
from datasets import load_dataset
import numpy as np


In [3]:
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-small-ade-semantic")
model


Mask2FormerForUniversalSegmentation(
  (model): Mask2FormerModel(
    (pixel_level_module): Mask2FormerPixelLevelModule(
      (encoder): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=96, out_features=96, bias=True)
                      (key): Linear(in_features=96, out_features=96, bias=True)
                      (value

In [4]:
dataset = load_dataset("nateraw/ade20k-tiny")
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 10
    })
})

In [17]:
image_processor = Mask2FormerImageProcessor.from_pretrained("facebook/mask2former-swin-small-ade-semantic")


In [18]:
# 定義資料前處理函數
def preprocess_function(examples):
    images = examples["image"]
    labels = examples["label"]
    if not isinstance(images, list):
        images = [images]

    processed_images = [image if isinstance(image, Image.Image) else Image.open(image) for image in images]
    processed_images = [image_processor.preprocess(image) for image in processed_images]
    if not isinstance(labels, list):
        labels = [labels]
    label_array = np.array([np.array(label) for label in labels])
    print(label_array)
    return {"image": processed_images, "label": label_array}

In [19]:
# 將資料轉換成Dataset格式
train_dataset = dataset["train"].map(preprocess_function)
eval_dataset = dataset["validation"].map(preprocess_function)

In [29]:
from transformers import default_data_collator

data_collator = default_data_collator

In [32]:
# 設定訓練參數
training_args = TrainingArguments(
    output_dir="./transformer_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    
    logging_dir="./logs",
    # label_names=["label"],
)

In [33]:
# 使用Trainer來訓練模型
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator = data_collator,
)


In [34]:
trainer.train()

  0%|          | 0/6 [00:00<?, ?it/s]

ValueError: expected sequence of length 512 at dim 2 (got 735)